# Scripts for MCDA

This is the entire script for the ordinary MCDA-analysis. Some preprocessing of the criteria are not included due to it being a pre-planning stage that is independent from MCDA. The work environment contains two geodatabases so far, one containing original data for the criteria "Intercity.gdb" and one specifically tailored for the study area "HaugSeut.gdb". 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import arcpy
from arcpy import env
from arcpy.sa import *

#### Set-up environment

In [2]:
arcpy.env.workspace = 'C:/Users/rmbp/GIS-project/Intercity_data.gdb/'
mydir = arcpy.env.workspace 
arcpy.env.overwriteOutput = True 

#### Euclidean Distance for pumping wells and rivers

In [3]:
# Assign local variables
insource_data = 'main_river.shp'
maxDistance = 6000
cellSize = 50
outdir = 'C:/Users/rmbp/GIS-project/Analysis/eucdirect'

# Checking ArcGIS Spatial Analyst
arcpy.CheckOutExtension('Spatial')

# Execute operation
out_Euclid = EucDistance(insource_data,maxDistance,cellSize,outdir)

# Save output
out_Euclid.save('C:/Users/rmbp/GIS-project/Analysis/euc_river.tif')

#Euclidean distance for pumping wells
new_insource = 'pumping_wells.shp'

outdir_pumps = 'C:/Users/rmbp/GIS-project/Analysis/eucdirect'
out_pumpdist = EucDistance(new_insource,maxDistance,cellSize,outdir_pumps)

# Save the output from the pumping wells 
out_pumpdist.save('C:/Users/rmbp/GIS-project/Analysis/euc_wells')


#### Clipping of study area

In [4]:
# Setting in raster data for extraction

#Polygon-mask 
inMask = 'HaugSeut'

# Raster-files for clipping the area 
inClass = 'classified_land'
inTWI = 'TWI_for_Intercity'
inSlope = 'Slope3'

outClass = ExtractByMask(inClass, inMask)
outTWI = ExtractByMask(inTWI, inMask)
outSlope = ExtractByMask(inSlope, inMask)

outClass.save('C:/Users/rmbp/GIS-project/HaugSeut.gdb/landcovers')
outTWI.save('C:/Users/rmbp/GIS-project/HaugSeut.gdb/TWI')
outSlope.save('C:/Users/rmbp/GIS-project/HaugSeut.gdb/Slope')

#Clipping of vector data
xy_tolerance = ''
in_soil = 'soil_type'
in_pump = 'GW_stablewater'

out_soil = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/soil_haugseut'
arcpy.Clip_analysis(in_soil,inMask,out_soil,xy_tolerance)

out_pump = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/pumps_haugseut'
arcpy.Clip_analysis(in_pump,inMask,out_pump,xy_tolerance)

<Result 'C:\\Users\\rmbp\\GIS-project\\HaugSeut.gdb\\pumps_haugseut'>

#### Union of deep weathering towards the study area and converting files to raster

In [5]:
# Union of mask and deep weathering

DW = 'deepweathering'
arcpy.Union_analysis([inMask,DW],'C:/Users/rmbp/GIS-project/HaugSeut.gdb/DW_unified')

# Clip to proper area
out_DW = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/DeepWeather'
arcpy.Clip_analysis('C:/Users/rmbp/GIS-project/HaugSeut.gdb/DW_unified',inMask,out_DW,xy_tolerance)

# Converting the polygon into raster
outRaster = 'C:/Users/rmbp/GIS-project/raster-files/DW-Raster'
valField = 'Dypforvitr'

insoil = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/soil_haugseut'
outsoil = 'C:/Users/rmbp/GIS-project/raster-files/soil-Raster'
valfieldsoil = 'jorda_navn'

arcpy.PolygonToRaster_conversion(out_DW,valField,outRaster,cell_assignment='MAXIMUM_AREA',cellsize=50)
arcpy.PolygonToRaster_conversion(insoil,valfieldsoil,outsoil,cell_assignment='MAXIMUM_AREA',cellsize=50)

<Result 'C:\\Users\\rmbp\\GIS-project\\raster-files\\soil-Raster'>

### Combining Multiconsult's piezometric wells and public well data together in a single shape-file

In [6]:
# Calling in values to merge, pumps_haigseut is the edited version
point_data = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/points'
GW_data = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/pumps_haugseut'

# Adding source data
addSourceInfo = 'ADD_SOURCE_INFO'

# Create FieldMappings object to manage merge output fields
fieldMappings = arcpy.FieldMappings()

# Adding fields to both GRANADA-pumping wells and pore pressure measurements from Haug-Seut
fieldMappings.addTable(point_data)
fieldMappings.addTable(GW_data)

# Add input fields "water table" and "vannstandb" into new output fields
fldMap_water = arcpy.FieldMap()
fldMap_water.addInputField(GW_data,'vannstandb')
fldMap_water.addInputField(point_data,'water_table')

# Set name of new output field as "stable water"
stable_water = fldMap_water.outputField
stable_water.name = 'Stable_Water'
fldMap_water.outputField = stable_water

# Add output field to fieldmappings object
fieldMappings.addFieldMap(fldMap_water)

# Finally merging data together
new_wells = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/new_wells_test'

# Removing unnecessary field attributes
for field in fieldMappings.fields:
    if field.name not in ['Stable_Water','OBJECTID','bronnnummer']:
        fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name))

# Start merging point data together
arcpy.Merge_management([point_data,GW_data],new_wells,fieldMappings,addSourceInfo)

<Result 'C:\\Users\\rmbp\\GIS-project\\HaugSeut.gdb\\new_wells_test'>

### Extracting altitude data to the piezometric well points and creating a field for altitude of water level

In [7]:
dtm = 'C:/Users/rmbp/GIS-project/Intercity_data.gdb/DEM_1m'
arcpy.env.workspace = 'C:/Users/rmbp/GIS-project/HaugSeut.gdb/'

# Saving an altitude map into the database
outDEM = ExtractByMask(dtm,'C:/Users/rmbp/GIS-project/Intercity_data.gdb/HaugSeut')
inDEM = 'Altitude'
outDEM.save(inDEM)

# Extract altitude values to points 
ExtractValuesToPoints('new_wells_test', 'Altitude',
                      'wells_with_alt')

# Creating new field for altitude of water table
in_features = 'wells_with_alt'
field_name = 'water_altitude'
expression = '!RASTERVALU! - !Stable_Water!'


arcpy.AddField_management(in_features, field_name, 
                          field_type = 'DOUBLE')

arcpy.CalculateField_management(in_features, field_name, 
                                expression)


<Result 'C:/Users/rmbp/GIS-project/HaugSeut.gdb\\wells_with_alt'>

#### Interpolation of piezometric well data (Need to be redone, make new table with respect to altitude, and then interpolate thedata)

In [8]:
# Use of Inverse Distance Weighting

# Assigning local variables
input_points = 'wells_with_alt'
zField = 'water_altitude'
outlay = 'IDW_wells'
outRast = 'IDW_RastWells'
cellSize = 50
power = 2


# Set up variables for search neighborhood
majSemiaxis = 300000
minSemiaxis = 300000
angle = 0
maxNeighbors = 15
minNeighbors = 10
sectorType = 'ONE_SECTOR'

searchNeigborhood = arcpy.SearchNeighborhoodStandard(majSemiaxis,minSemiaxis,angle,
                                                    maxNeighbors,minNeighbors,sectorType)
# Execute IDW 
arcpy.IDW_ga(input_points,zField,outlay,outRast)

<Result 'IDW_wells'>

#### Reclassify

Now that we have all data prepared. The values need to be reclassified for proper weighting process. We reclassify the classes in a scale from 1 to 10, where 10 stands for the highest rating for land subsidence susceptibility. We do this by changing from nominal values that represents a class to interval or ratio values so that values can be used in relation to one another. 

In [9]:
# Reclassifying parameters
inRiver = 'final_river'
inDeep = 'C:/Users/rmbp/GIS-project/raster-files/dw-raster'
inSlope = 'Slope'
inLULC = 'landcovers'
inTWI = 'TWI'
inSoil = 'C:/Users/rmbp/GIS-project/raster-files/soil-raster'
inGW = 'Wells'

# Remapping values
myRemapRiver = RemapRange([[0,100,8],[100,200,6],
                           [200,300,3],[300,6000,1]])

myRemapDW = RemapValue([[1,1],[2,9],[3,5],[4,4],[5,8],[6,4]])

myRemapSlope = RemapRange([[0,5,7],[5,15,6],[15,20,5],[20,40,2]])

myRemapLand = RemapValue([["Water","NODATA"],["Barren",2],["Planted/Cultivated",7],["Urban",8],
                          ["Forest",6],["Coastline","NODATA"]])

myRemapTWI = RemapRange([[0,6,2],[6,9,5],[9,12,6],[12,14,7],[14,30,9]])

myRemapSoil = RemapValue([[3,1],[1,10],[2,10],[6,10],[7,4],[9,4],[4,7],[5,7],[10,8],[11,6],[8,9]])

myRemapGW = RemapRange([[-67,-15,10],[-15,-5,8],[-5,15,6],[15,30,4],
                       [30,45,2],[45,70,1]])


# Execute river reclassify, you should note that it's possible to use a nested loop to run the anaysis
outReclassRiver = Reclassify(inRiver,"VALUE",myRemapRiver)
outReclassDW = Reclassify(inDeep,"VALUE",myRemapDW)
outReclassSlope = Reclassify(inSlope,"VALUE",myRemapSlope)
outReclassLand = Reclassify(inLULC,"Class_name",myRemapLand)
outReclassTWI = Reclassify(inTWI,"VALUE",myRemapTWI)
outReclassSoil = Reclassify(inSoil,"VALUE",myRemapSoil)
outReclassGW = Reclassify(inGW,"VALUE",myRemapGW)

# Save reclass-files to geodatabase containing 
outReclassRiver.save('C:/Users/rmbp/GIS-project/test_reclassify.gdb/reclass_riv')
outReclassDW.save('C:/Users/rmbp/GIS-project/test_reclassify.gdb/reclass_DW')
outReclassSlope.save('C:/Users/rmbp/GIS-project/test_reclassify.gdb/reclass_Slope')
outReclassLand.save('C:/Users/rmbp/GIS-project/test_reclassify.gdb/reclass_Land')
outReclassTWI.save('C:/Users/rmbp/GIS-project/test_reclassify.gdb/reclass_TWI')
outReclassSoil.save('C:/Users/rmbp/GIS-project/test_reclassify.gdb/reclass_Soil')
outReclassGW.save('C:/Users/rmbp/GIS-project/test_reclassify.gdb/reclass_GW')

N.B some of the classes are registered to NODATA. This accounts for "Coastline" in the landcover layer and "Unsure data" for the deep weathering layer. 

#### Analytical hierarchy process

For the next step, we need to apply Analytical Hierarchy process. Now, we have to measure the relative importance of the criteria to each other. Use the Python-module Pandas to read the Excel-file containing the pairwise comparison matrix to calculate the results. 

In [16]:
def AHP(n, sheet_no):
    
    # n: No of criteria
    # sheet_no: Sheet name in Excel
    # RI: Random Index
    
    # Reading file location of the 7x7 pairwise comparison matrix
    file_loc = "C:/Users/rmbp/GIS-project/Excel-files/AHP.xlsx"
    df = pd.read_excel(file_loc,sheet_name=sheet_no)
    df1 = df.drop(["Letter"],axis=1)
    
    # Normalizing the nth root of products to get appropriate weights
    root = df1.product(axis = 1)**(1/n)
    sum_val = sum(root)
    
    # The final priority vectors
    weights = root/sum_val
    
    # Calculating the consistency ratio
    if n == 7:
        col_list = ["A","B","C","D","E","F","G"]
        RI = 1.32
    elif n == 6:
        col_list = ["A","B","C","D","E","F"]
        RI = 1.24
    
    col_sum = []
    for i in col_list:
        col_sum.append(df[i].sum())
        
    lam_max = np.sum(col_sum*weights)
    CI = (lam_max - n)/(n - 1)
    CR = CI/RI
    
    return weights, CR

# Original weights
W1, C1 = AHP(7,"Sheet2")

# Removal of DW
W2, C2 = AHP(6,"Sheet3")

# Removal of Lithology
W3, C3 = AHP(6,"Sheet4")

# Removal of land cover
W4, C4 = AHP(6,"Sheet5")

print(W1)
print(C1,C2,C3,C4)

# Removal of remaining water in the map through raster calculator
# SetNull( ~ (IsNull( [EraseRaster] )), [OriginalRaster] )
# The weights will be applied in the overlay analysis
# Roads are counted as urban areas

0    0.165127
1    0.068511
2    0.335858
3    0.032950
4    0.243126
5    0.104140
6    0.050288
dtype: float64
0.06648233753402097 0.0851412662482208 0.04933523690973838 0.08964418876663842


### Extraction of most susceptible areas by InSAR

The overlay analysis is performed by using the raster calculator.

In [15]:
inSAR = "C:/Users/rmbp/GIS-project/main_env/MyProject1/MyProject1.gdb/Extract_sent1"
SQLClause = "VALUE >= 0.45"

attExtract = ExtractByAttributes(inSAR, SQLClause) 
attExtract.save('C:/Users/rmbp/GIS-project/Stage2.gdb/observed_subsidence')
